### Wasn't that awesome?
Ok! So Step 1 was great, right? We wrote 30 or 40 lines of Python! We got some stuff! 
Oh wait, we could have gotten the same result just copying and pasting into a spreadsheet. Sad trombone.

Now that we know how to scrape one page, though, we can add bells and whistles until we can scrape ALL the pages. This is a pretty common Python workflow: You break your problem into small, solvable pieces, then build around it until you have a script that solves the whole thing.

In this case, we'll start by finding a way to "click" through all the different pages with data for 2018. We're going to start with a bunch of the ~~junk~~ valuable, well-crafted code from the first section.



In [7]:
import requests
from bs4 import *
import csv

url = "https://www.fda.gov/ICECI/EnforcementActions/WarningLetters/2018/default.htm"
response = requests.get(url)

html = response.content
soup = BeautifulSoup(html, "html5lib")
table = soup.find('table')
pageno = 1
moreflag = True

list_of_rows = []

This should look mostly familiar. Again we're using `requests` and `bs4`, and again we'll write the information using `csv`. We'll use the repetitive requests/response/BeautifulSoup process, and access the table within that object.

But what's the deal with `pageno` and `moreflag`? Again taking a look at our [target page](https://www.fda.gov/ICECI/EnforcementActions/WarningLetters/2018/default.htm), we can see that every time we view the "next" page, the url tacks on a suffix with the number of the results page, "?Page=2" for the second page of results, "?Page=3" for the third, and so on. `pageno` is going to fill in that section with the correct page of results that we want to read. 

`moreflag`, then, is there to tell the scraper when to wrap it up. On the last page of results, the "Next" button that had been clickable on earlier pages suddenly disappears. We're going to set up our script to evaluate whether or not that link is clickable, then change this variable to False if the condition changes.

As you can tell, almost everything about this is pretty specific to the page we're trying to scrape. In this case, for example, we only have one table. If there were multiple tables, that would be yet another problem we'd have to solve. On this page, the url clearly changes to reflect the page of data we seek. On other pages, that could be hidden or nonexistent. *Every case is different and there's no "right" way to do anything.* Google around, hit stackoverflow, and reach out the the fantastic IRE/NICAR community ([including me](mailto:matt.wynn@gmail.com)if you're in a pickle. 

Anyhow. On with the scraper.

In [8]:
while moreflag is True:
    for row in table.findAll('tr')[1:]:
        list_of_cells = []
        for cell in row.findAll('td'):
            text = cell.text.encode("utf8").strip()
            list_of_cells.append(text)
        list_of_rows.append(list_of_cells)
    if len(soup.findAll('a', href=True, text="Next")) > 0:
        pageno+=1
        newurl = url + "?Page=" + str(pageno)
        response = requests.get(newurl)
        html = response.content
        soup = BeautifulSoup(html, "html5lib")
        table = soup.find('table')
    else:
         moreflag = False

A lot of this looks familiar. We're again stepping through each table row, again creating empty lists of rows and cells and them filling them with information. 

A few new controls are putting in a lot of work:
* `while moreflag is True:`. The biggest difference from our first go-round is that all of this is tucked within thie `while` loop. A for loop goes through a procedure for each thing in a list, then finishes. A while loop, meanwhile, keeps going so ong as a condition is met. That leads us to the second big difference...
* `if len(soup.findAll('a', href=True, text="Next")) > 0:`. This section is evaluting whether or not there's even another page to find. Using good ol' BeautifulSoup/bs4, we're tracking down the a link with the text "Next." The `len` bit is a trick to find out how many times we found such a link. Then, if there's more than 0, we've met the conditional of the `if` statement and go inside the loop. IF there is no such link, then our `else` condition comes into play. We set `moreflag` to False, which breaks out of our while loop, and we break out of this process entirely.
* `pageno+=1`. This one's neat. If there is indeed another page, we can use this neat trick to set `pageno` equal to `pageno` plus one. Using this handy trick, we can change our url to get the next page and read it in with `requests`, parse it with `BeautifulSoup`, and continue onto the next iteration.

Now let's write it to a csv, the exact same way as we did before.

In [9]:
outfile = open('fdaresults.txt', 'w')
writer = csv.writer(outfile)
writer.writerows(list_of_rows)
outfile.close()